In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import lightgbm as lgb
from pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from lightgbm import plot_importance
from datetime import datetime
%matplotlib inline

In [19]:
# load data
train = pd.read_csv("./train.csv", index_col=0)
test = pd.read_csv("./test.csv", index_col=0)
submission = pd.read_csv("./sample_submission.csv")

In [20]:
train = train.dropna()
train = train.drop(train[train['dered_g']==-9999].index)
train = train.drop(train[train['dered_i']==-9999].index)
train = train.drop(train[train['dered_z']==-9999].index)

In [21]:
#유용한 값들 추가
train['u-g'] = train['u'] - train['g']
train['g-r'] = train['g'] - train['r']
train['r-i'] = train['r'] - train['i']
train['i-z'] = train['i'] - train['z']
train['dered_u-g'] = train['dered_u'] - train['dered_g']
train['dered_g-r'] = train['dered_g'] - train['dered_r']
train['dered_r-i'] = train['dered_r'] - train['dered_i']
train['dered_i-z'] = train['dered_i'] - train['dered_z']

test['u-g'] = test['u'] - test['g']
test['g-r'] = test['g'] - test['r']
test['r-i'] = test['r'] - test['i']
test['i-z'] = test['i'] - test['z']
test['dered_u-g'] = test['dered_u'] - test['dered_g']
test['dered_g-r'] = test['dered_g'] - test['dered_r']
test['dered_r-i'] = test['dered_r'] - test['dered_i']
test['dered_i-z'] = test['dered_i'] - test['dered_z']

In [22]:
# # Magnitude m in terms of radiant flux F
# train['u/g'] = np.exp(-train['u-g'])
# train['g/r'] = np.exp(-train['g-r'])
# train['r/i'] = np.exp(-train['r-i'])
# train['i/z'] = np.exp(-train['i-z'])

In [23]:
# n값 처리
train['nO-nD'] = train['nObserve'] - train['nDetect']
del train['nObserve']
del train['nDetect']
del train['airmass_u']
del train['airmass_g']
del train['airmass_r']
del train['airmass_i']
del train['airmass_z']

test['nO-nD'] = test['nObserve'] - test['nDetect']
del test['nObserve']
del test['nDetect']
del test['airmass_u']
del test['airmass_g']
del test['airmass_r']
del test['airmass_i']
del test['airmass_z']

In [24]:
# rare_train = train[["u", "g", "r", "i", "z"]]
# dered_train = train[["dered_u", "dered_g", "dered_r", "dered_i", "dered_z"]]

# rare_test = test[["u", "g", "r", "i", "z"]]
# dered_test = test[["dered_u", "dered_g", "dered_r", "dered_i", "dered_z"]]

In [25]:
# train["rare_mean"] = rare_train.mean(axis = 1)
# train["rare_var"] = rare_train.var(axis = 1)
# train["rare_std"] = rare_train.std(axis = 1)
# train["rare_sum"] = rare_train.sum(axis = 1)
# train["rare_median"] = rare_train.median(axis = 1)
# train['dered_mean'] = dered_train.mean(axis=1)
# train['dered_var'] = dered_train.var(axis=1)
# train['dered_std'] = dered_train.std(axis=1)
# train['dered_sum'] = dered_train.sum(axis=1)
# train['dered_median'] = dered_train.median(axis=1)

# test["rare_mean"] = rare_test.mean(axis = 1)
# test["rare_var"] = rare_test.var(axis = 1)
# test["rare_std"] = rare_test.std(axis = 1)
# test["rare_sum"] = rare_test.sum(axis = 1)
# test["rare_median"] = rare_test.median(axis = 1)
# test['dered_mean'] = dered_test.mean(axis=1)
# test['dered_var'] = dered_test.var(axis=1)
# test['dered_std'] = dered_test.std(axis=1)
# test['dered_sum'] = dered_test.sum(axis=1)
# test['dered_median'] = dered_test.median(axis=1)

In [26]:
# split data into X and y
Y = train['class']
del train['class']
X = train

In [27]:
#train-test-split
X_train,X_test,y_train,y_test=train_test_split(X, Y, test_size=0.25, random_state=0)
d_train = lgb.Dataset(X_train, label=y_train)
eval_set = [(X_test, y_test)]

In [ ]:
start_time = datetime.now()

#setting up the parameters
params={}
params['learning_rate']=0.1
params['boosting_type']='gbdt'
params['objective']='multiclass'
params['metric']='multi_logloss'
params['num_class']=3
# params['scale_pos_weight']=1.2
params['max_depth']=11
# params['feature_fraction']=0.8
params['num_estimators']=1000

#training the model
model = lgb.train(params,d_train,1000)
y_pred = model.predict(X_test)

end_time = datetime.now()
print("\nelapsed time = ", end_time - start_time)

In [ ]:
plot_importance(model)

In [ ]:
#argmax() method 
y_pred = [np.argmax(line) for line in y_pred]

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
# pred = model.predict(test)
# pred = [np.argmax(line) for line in pred]
# submission['class'] = pred
# submission.to_csv("submission_lgb.csv",index=False, encoding='utf-8-sig')